**CS547 HW2**  
Yue Cui  
Gaoyu Liu


# Problem 1

$$
H(p',p) = p'\ln \frac{p'}{p}+(1-p')\ln \frac{1-p'}{1-p}
$$

Take the derivative of $H(p', p)$ w.r.t $p'$:

$$

$$

# Problem 2

# Problem 3

In [1]:
# What version of Python do you have?
import sys

import torch
import pandas as pd
import sklearn as sk
import numpy as np

print(f"PyTorch Version: {torch.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print("GPU is", "available" if torch.cuda.is_available() else "NOT AVAILABLE")

PyTorch Version: 1.7.1

Python 3.7.9 (default, Aug 31 2020, 17:10:11) [MSC v.1916 64 bit (AMD64)]
Pandas 1.2.1
Scikit-Learn 0.23.2
GPU is available


Generate the data.

In [32]:
# generate 200 Gausian points
torch.manual_seed(1)
N = 200

data_input = torch.Tensor(N, 1).normal_(mean = 0., std = 1.)
#print(data_input)

labels_input = (data_input > 0).float()
#print(labels_input)

# check for GPU
if torch.cuda.is_available():
    data_input = data_input.cuda()
    labels_input = labels_input.cuda()

## (1) 

Define the class **LogisticRegression** using Pytorch. The Sigmoid function 

$$
S_{m, b}\ (X) = \frac{e^{mX+b}}{1 + e^{mX+b}}
$$

is used and a entropy function for binary classification problem is adopted, namely, **torch.nn.BCEWithLogitsLoss**.  

For each training sample, the input $X$ is a scalar with dimension 1, the output $z$ is also a scalar with dimension of 1.  

The learning rate is set to be 0.1.

In [132]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        # https://pytorch.org/docs/stable/generated/torch.nn.Linear.html
        self.linear = torch.nn.Linear(input_size, num_classes)

    def forward(self, x):
        out = self.linear(x)
        return out


    
# flip n*2 points as needed (for (2) and (3))
def flip_n(data, labels, n):
    if n == 0:
        return labels.clone()
    # get the assending sorted indices
    sort_idx = data.argsort(dim=0)
    # get the total number of negative numbers
    num_neg = sum(data < 0)
    # get the indices to flip the labels
    flip_idx_neg = sort_idx[num_neg - n: num_neg]
    flip_idx_pos = sort_idx[num_neg: num_neg + n]
    # carry out the label flipping
    labels_res = labels.clone()
    labels_res[flip_idx_neg] = 1.
    labels_res[flip_idx_pos] = 0.
    # return the result
    return labels_res



def Carryout_logi_regression(data_input,
                             labels_input,
                             n=0,
                             lr=.1, max_iter=10000,
                             ):
    # process the labels
    labels_n = flip_n(data_input, labels_input, n)

    # instantiate the model object
    input_dim = 1  # data_raw.size()[0]
    num_classes = 1

    model = LogisticRegression(input_dim, num_classes)
    if torch.cuda.is_available():
        model.cuda()

    #Loss = torch.nn.CrossEntropyLoss()
    # https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html
    Loss = torch.nn.BCEWithLogitsLoss(reduction='mean')

    learningRate = lr
    optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)

    for itr in range(max_iter):
        # Clear gradient buffers because we don't want any gradient from previous epoch to carry forward,
        # dont want to cummulate gradients
        optimizer.zero_grad()

        # get output from the model, given the inputs
        outputs = model(data_input)
        # print(outputs.size())
        # print(labels_input.size())

        # get loss for the predicted output
        lossvalue = Loss(outputs, labels_n)

        # get gradients w.r.t to parameters
        lossvalue.backward()
        # print(model.linear.weight.grad.item(),model.linear.bias.grad.item())

        # update parameters
        optimizer.step()
        if itr % 10000 == 0:
            print("iteration {}: loss={:.5f}, width={:.5f} ".format(itr, lossvalue.item(),
                                                                    1/model.linear.weight.item()))
    #
    print(f'The width of the transition layer for the model with {n} points\n',
        f'flipped on each side of 0 is {1/model.linear.weight.item():.5f}')
    print(f'===============================================\n')

Now carry out the logistic regression.

In [134]:
Carryout_logi_regression(data_input,
                         labels_input,
                         n=0,
                         lr=0.1, max_iter=100000,
                         )

iteration 0: loss=0.54682, width=1.46370 
iteration 10000: loss=0.05050, width=0.08271 
iteration 20000: loss=0.03936, width=0.06494 
iteration 30000: loss=0.03400, width=0.05648 
iteration 40000: loss=0.03066, width=0.05120 
iteration 50000: loss=0.02831, width=0.04748 
iteration 60000: loss=0.02654, width=0.04467 
iteration 70000: loss=0.02514, width=0.04242 
iteration 80000: loss=0.02400, width=0.04058 
iteration 90000: loss=0.02303, width=0.03903 
The width of the transition layer for the model with 0 points
 flipped on each side of 0 is 0.03770



## (2)

In [137]:
Carryout_logi_regression(data_input,
                         labels_input,
                         n=5,
                         lr=.1, max_iter=100000,
                         )

iteration 0: loss=0.77725, width=-6.59151 
iteration 10000: loss=0.07548, width=0.09069 
iteration 20000: loss=0.06993, width=0.07495 
iteration 30000: loss=0.06804, width=0.06800 
iteration 40000: loss=0.06718, width=0.06399 
iteration 50000: loss=0.06673, width=0.06138 
iteration 60000: loss=0.06649, width=0.05957 
iteration 70000: loss=0.06634, width=0.05826 
iteration 80000: loss=0.06626, width=0.05728 
iteration 90000: loss=0.06620, width=0.05653 
The width of the transition layer for the model with 5 points
 flipped on each side of 0 is 0.05594



## (3)

In [138]:
for item in [15, 20, 25, 30, 35]:
    Carryout_logi_regression(data_input,
                             labels_input,
                             n=item,
                             lr=.001, max_iter=100000,
                             )    

iteration 0: loss=0.77496, width=-5.27807 
iteration 10000: loss=0.31211, width=0.57211 
iteration 20000: loss=0.25764, width=0.40385 
iteration 30000: loss=0.23543, width=0.33955 
iteration 40000: loss=0.22327, width=0.30370 
iteration 50000: loss=0.21564, width=0.28024 
iteration 60000: loss=0.21045, width=0.26343 
iteration 70000: loss=0.20672, width=0.25069 
iteration 80000: loss=0.20393, width=0.24063 
iteration 90000: loss=0.20179, width=0.23246 
The width of the transition layer for the model with 15 points
 flipped on each side of 0 is 0.22566

iteration 0: loss=0.55553, width=2.39460 
iteration 10000: loss=0.32448, width=0.54613 
iteration 20000: loss=0.28679, width=0.41034 
iteration 30000: loss=0.27165, width=0.35412 
iteration 40000: loss=0.26379, width=0.32225 
iteration 50000: loss=0.25916, width=0.30141 
iteration 60000: loss=0.25622, width=0.28663 
iteration 70000: loss=0.25426, width=0.27558 
iteration 80000: loss=0.25290, width=0.26701 
iteration 90000: loss=0.25193, 